In [21]:
import ipyparallel
# import numpy as np
# import os
# import pandas as pd
import sys
MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, MODULE_PATH)
from custom_module.utilities import *

In [2]:
# FMA
# collect track id and genres of tracks in the small subset.
print('collecting track id and genres of tracks in the small subset of fma dataset')
tracks = load(MOUNTED_DATASET_PATH + '/fma_metadata/tracks.csv')
fma_full = tracks[[('set', 'subset'), ('track', 'genre_top')]]
small_subset = fma_full[('set', 'subset')] == 'small'
fma_small = fma_full[small_subset]
fma_small = pd.DataFrame({
    'subset': fma_small[('set', 'subset')],
    'label': fma_small[('track', 'genre_top')]
})
print('done')

collecting track id and genres of tracks in the small subset of fma dataset
done


In [3]:
# attach to a running cluster to notebook
cluster = ipyparallel.Client()
# print profile name and process id numbers
print('profile:', cluster.profile)
print("IDs:", cluster.ids)

profile: default
IDs: [0, 1, 2, 3, 4, 5, 6, 7]


In [4]:
# split into tow set of 4 engines each and get DirectView object of each set.
direct_views_first_4_engines = cluster[:4]
direct_views_last_4_engines = cluster[5:]

In [5]:
# sync import for each DirectView object
with direct_views_first_4_engines.sync_imports():
    remote_imports()

importing ast on engine(s)
importing CategoricalDtype from pandas.api.types on engine(s)
importing ipyparallel on engine(s)
importing librosa on engine(s)
importing numpy on engine(s)
importing os on engine(s)
importing pandas on engine(s)
importing pywt on engine(s)
importing skew from scipy.stats on engine(s)


In [6]:
with direct_views_last_4_engines.sync_imports():
    remote_imports()

importing ast on engine(s)
importing CategoricalDtype from pandas.api.types on engine(s)
importing ipyparallel on engine(s)
importing librosa on engine(s)
importing numpy on engine(s)
importing os on engine(s)
importing pandas on engine(s)
importing pywt on engine(s)
importing skew from scipy.stats on engine(s)


In [7]:
# initialize shared resouces for a DirectView object
direct_views_first_4_engines['MOUNTED_DATASET_PATH'] = MOUNTED_DATASET_PATH
direct_views_first_4_engines['LOCAL_MOUNTED_DATASET_PATH'] = LOCAL_MOUNTED_DATASET_PATH
direct_views_first_4_engines['SAMPLE_FILE'] = SAMPLE_FILE
direct_views_first_4_engines['GENRES'] = GENRES
direct_views_first_4_engines['extract_audio_features'] = extract_audio_features
direct_views_first_4_engines['dataframe'] = dataframe
direct_views_first_4_engines['fma_small'] = fma_small

In [8]:
direct_views_last_4_engines['MOUNTED_DATASET_PATH'] = MOUNTED_DATASET_PATH
direct_views_last_4_engines['LOCAL_MOUNTED_DATASET_PATH'] = LOCAL_MOUNTED_DATASET_PATH
direct_views_last_4_engines['SAMPLE_FILE'] = SAMPLE_FILE
direct_views_last_4_engines['GENRES'] = GENRES
direct_views_last_4_engines['extract_audio_features'] = extract_audio_features
direct_views_last_4_engines['dataframe'] = dataframe
direct_views_last_4_engines['fma_small'] = fma_small

In [9]:
# activate DirectView object and set suffix
direct_views_first_4_engines.activate('_first')
direct_views_last_4_engines.activate('_last')

In [13]:
dataframe_gtzan = dataframe.copy()
dataframe_fma = dataframe.copy()

In [30]:
# %%px_first
# GTZAN
# extract features

print('extracting features from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        genre_label = str(file.name).split('.')[0]
        if genre_label in GENRES:
            dataframe_gtzan = extract_audio_features(dataframe_gtzan, file, genre_label, 'gtzan')
            
dataframe_gtzan.to_csv('data/features_gtzan.csv')
print('done')

extracting features from gtzan dataset
appended features extracted from hiphop.00000.wav with genre: hiphop
appended features extracted from hiphop.00001.wav with genre: hiphop
appended features extracted from hiphop.00002.wav with genre: hiphop
appended features extracted from hiphop.00003.wav with genre: hiphop
appended features extracted from hiphop.00004.wav with genre: hiphop
appended features extracted from hiphop.00005.wav with genre: hiphop
appended features extracted from hiphop.00006.wav with genre: hiphop
appended features extracted from hiphop.00007.wav with genre: hiphop
appended features extracted from hiphop.00008.wav with genre: hiphop
appended features extracted from hiphop.00009.wav with genre: hiphop
appended features extracted from hiphop.00010.wav with genre: hiphop
appended features extracted from hiphop.00011.wav with genre: hiphop
appended features extracted from hiphop.00012.wav with genre: hiphop
appended features extracted from hiphop.00013.wav with genre: hi

appended features extracted from pop.00020.wav with genre: pop
appended features extracted from pop.00021.wav with genre: pop
appended features extracted from pop.00022.wav with genre: pop
appended features extracted from pop.00023.wav with genre: pop
appended features extracted from pop.00024.wav with genre: pop
appended features extracted from pop.00025.wav with genre: pop
appended features extracted from pop.00026.wav with genre: pop
appended features extracted from pop.00027.wav with genre: pop
appended features extracted from pop.00028.wav with genre: pop
appended features extracted from pop.00029.wav with genre: pop
appended features extracted from pop.00030.wav with genre: pop
appended features extracted from pop.00031.wav with genre: pop
appended features extracted from pop.00032.wav with genre: pop
appended features extracted from pop.00033.wav with genre: pop
appended features extracted from pop.00034.wav with genre: pop
appended features extracted from pop.00035.wav with gen

appended features extracted from rock.00049.wav with genre: rock
appended features extracted from rock.00050.wav with genre: rock
appended features extracted from rock.00051.wav with genre: rock
appended features extracted from rock.00052.wav with genre: rock
appended features extracted from rock.00053.wav with genre: rock
appended features extracted from rock.00054.wav with genre: rock
appended features extracted from rock.00055.wav with genre: rock
appended features extracted from rock.00056.wav with genre: rock
appended features extracted from rock.00057.wav with genre: rock
appended features extracted from rock.00058.wav with genre: rock
appended features extracted from rock.00059.wav with genre: rock
appended features extracted from rock.00060.wav with genre: rock
appended features extracted from rock.00061.wav with genre: rock
appended features extracted from rock.00062.wav with genre: rock
appended features extracted from rock.00063.wav with genre: rock
appended features extract

In [23]:
# %%px_first --targets 1 --noblock
# FMA
# extract features

print('extracting features from fma dataset')

for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():
                track_id = int(file.name[:-4].lstrip('0'))
                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')
                if genre_label in GENRES:
                    dataframe_fma = extract_audio_features(dataframe_fma, file, genre_label, 'fma')
                    
dataframe_fma.to_csv('data/features_fma.csv')
print(done)

<AsyncResult: execute>

In [41]:
# check size of  log-mel / constant-Q transform in gtzan

# create an empty list to store sizes in
sizes = []
print('extracting log-mel from gtzan dataset')
for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label
        genre_label = str(file.name).split('.')[0]
        if genre_label in GENRES:
            scaled_cqt = extract_cqt(file)
            # adding the size to the list
            sizes.append(scaled_cqt.shape)

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))
# check the max size
print('The maximum size is: ' + str(max(sizes)))

extracting log-mel from gtzan dataset


/home/macbookretina/.local/lib/python3.5/site-packages/librosa/core/spectrum.py:1702: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn('amplitude_to_db was called on complex input so phase '


The sizes of all the log-mel in our data set are equal: False
The maximum size is: (84, 1320)


In [42]:
# check size of mel-spectogram in gtzan

# create an empty list to store sizes in
sizes_2 = []
print('extracting mel-spectogram from gtzan dataset')
for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label
        genre_label = str(file.name).split('.')[0]
        if genre_label in GENRES:
            scaled_mel_spect = extract_mel_spect(file)
            # adding the size to the list
            sizes.append(scaled_mel_spect.shape)
    
# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))
# check the max size
print('The maximum size is: ' + str(max(sizes)))

extracting mel-spectogram from gtzan dataset
The sizes of all the log-mel in our data set are equal: False
The maximum size is: (128, 1320)


In [24]:
%%px_first --targets 2 --noblock
# check size of  log-mel / constant-Q transform in fma
    
# creating an empty list to store sizes in
sizes = []
print('extracting log-mel from fma dataset')
for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():
                # extract track id
                track_id = int(file.name[:-4].lstrip('0'))
                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')
                if genre_label in GENRES:
                    scaled_cqt = extract_cqt(file)
                    # adding the size to the list
                    sizes.append(scaled_cqt.shape)

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))
# check the max size
print('The maximum size is: ' + str(max(sizes)))

<AsyncResult: execute>

In [25]:
%%px_first --targets 3 --noblock
# check size of  mel-spectogram  in fma

# creating an empty list to store sizes in
sizes = []
print('extracting mel-spectogram  from fma dataset')
for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():
                # extract track id
                track_id = int(file.name[:-4].lstrip('0'))
                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')
                if genre_label in GENRES:
                    scaled_mel_spect = extract_mel_spect(file)
                    # adding the size to the list
                    sizes.append(scaled_mel_spect.shape)                

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))
# check the max size
print('The maximum size is: ' + str(max(sizes)))

<AsyncResult: execute>

In [ ]:
%%px_last --targets 4 --noblock
# extract log-mel / constant-Q transform in gtzan

# create an empty list to store extract feature and label
cqts = []
genre_labels_1 = []
data_sources_1 = []

print('extracting log-mel from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label
        genre_label = str(file.name).split('.')[0]
        
        if genre_label in GENRES:
            # append to list
            genre_labels_1.append(genre_label)
            data_sources_1.append('gtzan')

            scaled_cqt = extract_cqt(file)

            # adjust the size to (84, 1320) as it is the max size
            if scaled_cqt.shape[1] != 1320:
                scaled_cqt.resize(84, 1320, refcheck=False)

            # flatten to fit into dataframe and add to the list
            scaled_cqt = scaled_cqt.flatten()
            cqts.append(scaled_cqt)

for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():

                # extract track id and map track id to genre label
                track_id = int(file.name[:-4].lstrip('0'))
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')

                if genre_label in GENRES:
                    # append to list
                    genre_labels_1.append(genre_label)
                    data_sources.append('fma')
                    
                    scaled_cqt = extract_cqt(file)

                    # adjust the size to (84, 1320) as it is the max size
#                     if scaled_cqt.shape[1] != 1320:
#                         scaled_cqt.resize(84, 1320, refcheck=False)

#                     # flatten to fit into dataframe and add to the list
#                     scaled_cqt = scaled_cqt.flatten()
#                     cqts.append(scaled_cqt)
        
# convert the lists to arrays so it can be stacked
# cqts = np.array(cqts)
# genre_labels = np.array(genre_labels).reshape(1000, 1)
# data_sources = np.array(data_sources).reshape(1000, 1)

# # create dataframe
# cqt_df = pd.DataFrame(np.hstack((genre_labels, data_sources, cqts)))
# cqt_df.to_csv('data/cqt.csv')

In [ ]:
%%px_last --targets 5 --noblock
# extract mel-spectogram in gtzan

# create an empty list to store extract feature and label
mel_spects = []
genre_labels_2 = []
data_sources_2 = []

print('extracting mel-spectogram from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label and append to list
        genre_label = str(file.name).split('.')[0]
        genre_labels_2.append(genre_label)
        
        if genre_label in GENRES:
            # append to list
            genre_labels_2.append(genre_label)
            data_sources_2.append('gtzan')
        
            scaled_mel_spect = extract_mel_spect(file)

            # adjust the size to (128, 1320) as it is the max size
            if scaled_mel_spect.shape[1] != 1320:
                scaled_mel_spect.resize(128, 1320, refcheck=False)

            # flatten to fit into dataframe and add to the list
            scaled_mel_spect = scaled_mel_spect.flatten()
            mel_spects.append(scaled_mel_spect)
            
for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():

                # extract track id and map track id to genre label
                track_id = int(file.name[:-4].lstrip('0'))
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')

                if genre_label in GENRES:
                    # append to list
                    genre_labels_2.append(genre_label)
                    data_sources_2.append('fma')

                    scaled_mel_spect = extract_mel_spect(file)

                    # adjust the size to (128, 1320) as it is the max size
#                     if scaled_mel_spect.shape[1] != 1320:
#                         scaled_mel_spect.resize(128, 1320, refcheck=False)

#                     # flatten to fit into dataframe and add to the list
#                     scaled_mel_spect = scaled_mel_spect.flatten()
#                     mel_spects.append(scaled_mel_spect)    
                    
# convert the lists to arrays so it can be stacked
# mel_spects = np.array(mel_spects)
# genre_labels = np.array(genre_labels).reshape(1000, 1)

# # create dataframe
# mel_spect_df = pd.DataFrame(np.hstack((genre_labels, mel_spects)))
# mel_spect_df.to_csv('data/mel_spect.csv')

In [26]:
%pxresult_first

KeyboardInterrupt: 

In [ ]:
%pxresult_last